# T1609 - Container Administration Command
Adversaries may abuse a container administration service to execute commands within a container. A container administration service such as the Docker daemon, the Kubernetes API server, or the kubelet may allow remote management of containers within an environment.(Citation: Docker Daemon CLI)(Citation: Kubernetes API)(Citation: Kubernetes Kubelet)

In Docker, adversaries may specify an entrypoint during container deployment that executes a script or command, or they may use a command such as <code>docker exec</code> to execute a command within a running container.(Citation: Docker Entrypoint)(Citation: Docker Exec) In Kubernetes, if an adversary has sufficient permissions, they may gain remote execution in a container in the cluster via interaction with the Kubernetes API server, the kubelet, or by running a command such as <code>kubectl exec</code>.(Citation: Kubectl Exec Get Shell)

## Atomic Tests

### Atomic Test #1 - ExecIntoContainer
Attackers who have permissions, can run malicious commands in containers in the cluster using exec command (“kubectl exec”). In this method, attackers can use legitimate images, such as an OS image (e.g., Ubuntu) as a backdoor container, and run their malicious code remotely by using “kubectl exec”.

**Supported Platforms:** containers

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: kubectl must be installed

##### Check Prereq Commands:
```bash
which kubectl

```
##### Get Prereq Commands:
```bash
echo "kubectl must be installed manually"

```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
kubectl create -f $PathtoAtomicsFolder/T1609/src/busybox.yaml -n default
# wait 3 seconds for the instance to come up
sleep 3
kubectl exec -n default busybox -- uname
```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 1

#### Cleanup: 
```bash
kubectl delete pod busybox -n default
```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Docker Exec Into Container
Attackers who have permissions, can run malicious commands in containers in the cluster using exec command (“docker exec”). In this method, attackers can use legitimate images, such as an OS image (e.g., Ubuntu) as a backdoor container, and run their malicious code remotely by using “docker exec”. Kinsing (Golang-based malware) was executed with an Ubuntu container entry point that runs shell scripts.

**Supported Platforms:** containers

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: docker must be installed

##### Check Prereq Commands:
```bash
which docker

```
##### Get Prereq Commands:
```bash
if [ "" == "`which docker`" ]; then echo "Docker Not Found"; if [ -n "`which apt-get`" ]; then sudo apt-get -y install docker ; elif [ -n "`which yum`" ]; then sudo yum -y install docker ; fi ; else echo "Docker installed"; fi

```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
docker build -t t1609  $PathtoAtomicsFolder/T1609/src/ 
docker run --name t1609_container --rm -itd t1609 bash /tmp/script.sh
docker exec -i t1609_container bash -c "cat /tmp/output.txt"
```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 2

#### Cleanup: 
```bash
docker stop t1609_container
docker rmi -f t1609:latest 
```

In [ ]:
Invoke-AtomicTest T1609 -TestNumbers 2 -Cleanup

## Detection
Container administration service activities and executed commands can be captured through logging of process execution with command-line arguments on the container and the underlying host. In Docker, the daemon log provides insight into events at the daemon and container service level. Kubernetes system component logs may also detect activities running in and out of containers in the cluster. 